## **Configurações iniciais**

In [ ]:
# Importa as bibliotecas que serão utilizadas na análise.

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Lê o .csv que contém os reviews e os salva em um DataFrame.
data = pd.read_csv("beer_reviews.csv")

In [ ]:
# Exibe o DataFrame gerado na célula anterior.
data

# **Filtragem dos dados**

## Processo

### Metodologia

Como o dataset *"beer_reviews.csv"* não contém uma coluna específica para nacionalidade, foi necessário realizar uma filtragem dos dados para que algumas cervejas brasileiras pudessem ser analisadas. Nesse intuito, foi consultada uma pesquisa publicada pelo Sindicerv (Sindicato Nacional da Indústria da Cerveja) para identificar os estilos de cerveja mais consumidos no Brasil. Em seguida, foram utilizados mecanismos de pesquisa para encontrar algumas cervejas nacionais que se enquadram em cada um desses estilos. Por fim, os dados de tais cervejas foram coletados e filtrados a partir do dataset "beer_reviews.csv" para possibilitar uma análise detalhada. 

*O processo completo dessa "filtragem de dados" está descrito nas células abaixo.*

### Pesquisa do site "Surra de Lúpulo" em conjunto com o Sindicato Nacional da Industria da Cerveja

Segundo a pesquisa "Retrato dos consumidores de Cerveja 2023", divulgada em dezembro de 2023 pelo site "Surra de Lúpulo", juntamente do Sindicato Nacional da Industria da Cerveja (Sindicerv), os estilos de cerveja mais consumidos pelo brasileiro são:

1. *IPA (India Pale Ale)*
2. *Weissbier*
3. *Pilsen*
4. *American Lager*
5. *APA (American Pale Ale)*

*Link da pesquisa:* https://www.sindicerv.com.br/noticias/ipa-pilsen-e-mais-pesquisa-revela-os-estilos-de-cervejas-preferidos-dos-consumidores/  

### Resumo do resultado das buscas nos mecanismos de pesquisa

###### *AKA Google + ChatGPT*

Para cada um dos estilos de cerveja mencionados, há representantes bem conhecidos no Brasil. Para os apreciadores de IPA, a Colorado Indica é uma excelente escolha. Se você gosta de Weissbier, a Eisenbahn Weizen é uma ótima opção. No que diz respeito às Pilsners, é comum que no Brasil elas sejam confundidas com American Lagers. Entre essas categorias, há diversas opções amplamente consumidas, como Brahma, Skol e Antártica. Finalmente, para quem prefere APA, a Colorado Appia é uma ótima opção.

### Obtendo dados das cervejas descobertas na pesquisa realizada na subseção anterior

#####
Com o intuito de facilitar o trabalho que será realizado nessa subseção, vamos criar um dicionário que guarda os estilos de cerveja mais consumidos pelos brasileiros, juntamente de algumas cervejas que representam tais estilos *(isto é, as cervejas que descobrimos na busca realizada na seção anterior)*.

In [ ]:
# Cria um dicionário que contém os estilos de cerveja mais consumidos pelos brasileiros. Cada estilo de cerveja tem uma ou mais cervejas 
# como suas representantes, e cada uma dessas cervejas possui um DataFrame que armazena os reviews relacionados a ela ou, no caso de algumas 
# cervejas, a ela e suas variantes.

beer_dict = {
    'India Pale Ale':{
        'Colorado Indica':pd.DataFrame()
    },
    'Weissbier':{
        'Eisenbahn Weizenbier':pd.DataFrame()
    },
    'Pilsen/American Lager':{
        'Skol':pd.DataFrame(),
        'Antarctica':pd.DataFrame(),
        'Brahma':pd.DataFrame()
    },
    'American Pale Ale':{
        'Colorado Appia':pd.DataFrame()
    }
}            

#### IPA (India Pale Ale)

In [ ]:
# Nesta célula serão exibidos os reviews sobre a cerveja Colorado Indica.

# Guarda os reviews sobre a cerveja em questão no dicionário "beer_dict".
beer_dict['India Pale Ale']['Colorado Indica'] = data[data['beer_name'] == 'Colorado Indica']

# Exibe tais reviews.
beer_dict['India Pale Ale']['Colorado Indica']

#### Weissbier

In [ ]:
# Nesta célula serão exibidos os reviews sobre a cerveja Eisenbahn Weizen.

# Guarda os reviews sobre a cerveja em questão no dicionário "beer_dict".
beer_dict['Weissbier']['Eisenbahn Weizenbier'] = data[data['beer_name'] == "Eisenbahn Weizenbier"]

# Exibe tais reviews.
beer_dict['Weissbier']['Eisenbahn Weizenbier']

#### Pilsen / American Lager

##### Skol

In [ ]:
# Nesta célula serão exibidos os reviews sobre a cerveja Skol.

# Perceba que nessa célula estamos usando o "startswith" para pesquisar sobre os reviews da cerveja em questão, ao invês de pesquisar 
# somente pelo nome de tal cerveja. Estamos fazendo isso pois existem várias variações da cerveja pesquisada.

# Guarda os reviews sobre as variantes da cerveja em questão no dicionário "beer_dict".
beer_dict['Pilsen/American Lager']['Skol'] = data[data['beer_name'].str.startswith("Skol", na=False)]

# Exibe tais reviews.
beer_dict['Pilsen/American Lager']['Skol']

# Observação: Veja que existem cervejarias que não são brasileiras e que produzem cervejas que começam com "Skol", resolveremos isso mais abaixo.

##### Antarctica

In [ ]:
# Nesta célula serão exibidos os reviews sobre a cerveja Antarctica.

# Perceba que nessa célula estamos usando o "startswith" para pesquisar sobre os reviews da cerveja em questão, ao invês de pesquisar 
# somente pelo nome de tal cerveja. Estamos fazendo isso pois existem várias variações da cerveja pesquisada.

# Guarda os reviews sobre as variantes da cerveja em questão no dicionário "beer_dict".
beer_dict['Pilsen/American Lager']['Antarctica'] = data[data['beer_name'].str.startswith("Antarctica", na=False)]

# Exibe tais reviews.
beer_dict['Pilsen/American Lager']['Antarctica']


##### Brahma

In [ ]:
# Nesta célula serão exibidos os reviews sobre a cerveja Brahma.

# Perceba que nessa célula estamos usando o "startswith" para pesquisar sobre os reviews da cerveja em questão, ao invês de pesquisar somente 
# pelo nome de tal cerveja. Estamos fazendo isso pois existem várias variações da cerveja pesquisada.

# Guarda os reviews sobre as variantes da cerveja em questão no dicionário "beer_dict".
beer_dict['Pilsen/American Lager']['Brahma'] = data[data['beer_name'].str.startswith("Brahma", na=False)]

# Exibe tais reviews.
beer_dict['Pilsen/American Lager']['Brahma']

# Observação: Tal como no caso da "Skol", a "Brahma" também possui cervejarias que não são nacionais e que produzem cerveja com as iniciais 
#             "Brahma", resolveremos isso na próxima seção.

#### APA (American Pale Ale)

##### Colorado Appia

In [ ]:
# Nesta célula serão exibidos os reviews sobre a cerveja Colorado Appia.

# Guarda os reviews sobre a cerveja em questão no dicionário "beer_dict".
beer_dict['American Pale Ale']['Colorado Appia'] = data[data['beer_name'] == 'Colorado Appia']

# Exibe tais reviews.
beer_dict['American Pale Ale']['Colorado Appia']

### Filtrando pelas cervejarias nacionais

Nessa subseção, resolveremos os problemas citados ao obter os dados das cervejas "Skol" e "Brahma".

##### Skol

In [ ]:
# Quem produz a cerveja 'Skol' (Pilsen / American Lager) ?

data[data['beer_name'].str.startswith('Skol')]['brewery_name'].unique()

# Observe que das cervejarias abaixo, apenas a "Cervejarias Reunidas Skol-Caracu" é brasileira.

In [ ]:
# Filtra os dados para obter apenas as variantes da cerveja "Skol" que foram produzidas pela cervejaria brasileira "Cervejarias Reunidas Skol-Caracu".
filtered_data = beer_dict['Pilsen/American Lager']['Skol'][beer_dict['Pilsen/American Lager']['Skol']['brewery_name'] == 'Cervejarias Reunidas Skol-Caracu']

# Altera o DataFrame da cerveja "Skol" para que este contenha apenas os dados filtrados que foram obtidos acima.
beer_dict['Pilsen/American Lager']['Skol'] = filtered_data

# Exibe os dados filtrados.
beer_dict['Pilsen/American Lager']['Skol']

In [ ]:
# Por fim, observe quais variantes da cerveja "Skol" são produzidas pela "Cervejarias Reunidas Skol-Caracu"

data[data['brewery_name'] == 'Cervejarias Reunidas Skol-Caracu']['beer_name'].unique()

##### Brahma

In [ ]:
# Quem produz a cerveja 'Brahma' (Pilsen / American Lager) ?

data[data['beer_name'].str.startswith('Brahma',na=False)]['brewery_name'].unique()

# As pesquisas são um pouco confusas e não sei ao certo afirmar se o Grupo Bemberg realmente produz Brahma. Por conta disso, vamos considerar
# que apenas a "Companhia Cervejaria Brahma" é a cervejaria nacional que produz a cerveja "Brahma" e suas variantes.

In [ ]:
# Filtra os dados para obter apenas as variantes da cerveja "Brahma" que foram produzidas pela cervejaria brasileira "Companhia Cervejaria Brahma".
filtered_data = beer_dict['Pilsen/American Lager']['Brahma'][beer_dict['Pilsen/American Lager']['Brahma']['brewery_name'] == 'Companhia Cervejaria Brahma']

# Altera o DataFrame da cerveja "Brahma" para que este contenha apenas os dados filtrados que foram obtidos acima.
beer_dict['Pilsen/American Lager']['Brahma'] = filtered_data

# Exibe os dados filtrados.
beer_dict['Pilsen/American Lager']['Brahma']

In [ ]:
# Por fim, observe quais variantes da cerveja "Brahma" são produzidas pela "Companhia Cervejaria Brahma"

data[data['brewery_name'] == 'Companhia Cervejaria Brahma']['beer_name'].unique()

### Escolhendo as cervejas que serão analisadas

#####

Cervejas do estilo "American Pale Ale", "Weissbier" e "India Pale Ale"

In [ ]:
# No caso do estilo "American Pale Ale", só temos como cerveja no "beer_dict" a "Colorado Appia". Portanto, não há o que alterar aqui.
beer_dict['American Pale Ale']['Colorado Appia']['beer_name'].unique()

In [ ]:
# Pelo mesmo motivo da célula anteiror, não há o que alterar aqui.
beer_dict['Weissbier']['Eisenbahn Weizenbier']['beer_name'].unique()

In [ ]:
# Pelo mesmo motivo das duas últimas células, não há o que alterar aqui.
beer_dict['India Pale Ale']['Colorado Indica']['beer_name'].unique()

#####
Cervejas do estilo "Pilsen/American Lager"

No caso das cervejas que seguem os estilos "Pilsen/American Larger", todas as suas representantes possuem variantes, veja:

In [ ]:
# Variantes da cerveja "Antarctica"

beer_dict['Pilsen/American Lager']['Antarctica']['beer_name'].unique()

In [ ]:
# Variantes da cerveja "Brahma"

beer_dict['Pilsen/American Lager']['Brahma']['beer_name'].unique()

In [ ]:
# Variantes da cerveja "Skol"

beer_dict['Pilsen/American Lager']['Skol']['beer_name'].unique()

Creio que das variantes acima, as mais conhecidas sejam a "Antarctica Pilsen", a "Brahma Chopp" e a "Skol".

Por conta disso, focaremos a análise das cervejas do tipo "Pilsen/American Lager" nessas 3 variantes. 

Dessa forma, vamos filtrar novamente os dados presentes em *"beer_dict"*.

In [ ]:
# Filtra os dados da cerveja "Skol" para eliminar todas as variantes, com exceção da variante "Skol".
filtered_skol_data = beer_dict['Pilsen/American Lager']['Skol'][beer_dict['Pilsen/American Lager']['Skol']['beer_name'] == 'Skol']
# Guarda os dados filtrados em "beer_dict".
beer_dict['Pilsen/American Lager']['Skol'] = filtered_skol_data
# Exibe os dados filtrados.
beer_dict['Pilsen/American Lager']['Skol']

In [ ]:
# Filtra os dados da cerveja "Brahma" para eliminar todas as variantes, com exceção da variante "Brahma Chopp".
filtered_brahma_data = beer_dict['Pilsen/American Lager']['Brahma'][beer_dict['Pilsen/American Lager']['Brahma']['beer_name'] == 'Brahma Chopp']
# Guarda os dados filtrados em "beer_dict".
beer_dict['Pilsen/American Lager']['Brahma'] = filtered_brahma_data
# Exibe os dados filtrados.
beer_dict['Pilsen/American Lager']['Brahma']

In [ ]:
# Filtra os dados da cerveja "Antarctica" para eliminar todas as variantes, com exceção da variante "Antarctica Pilsen".
filtered_antarctica_data = beer_dict['Pilsen/American Lager']['Antarctica'][beer_dict['Pilsen/American Lager']['Antarctica']['beer_name'] == 'Antarctica Pilsen']
# Guarda os dados filtrados em "beer_dict".
beer_dict['Pilsen/American Lager']['Antarctica'] = filtered_antarctica_data
# Exibe os dados filtrados.
beer_dict['Pilsen/American Lager']['Antarctica']

### Criando um DataFrame com os dados filtrados

*O DataFrame resultante dessa subseção será elaborado com base em tudo o que foi discutido nas subseções anteriores da seção "Filtragem dos dados"*

In [ ]:
# Cria uma lista auxiliar que conterá todos os DataFrames presentes no dicionário "beer_dict".
dfs = []

# Itera sobre cada estilo de cerveja no dicionário "beer_dict".
for style, beers in beer_dict.items():
    # Itera sobre as cervejas de cada estilo.
    for beer_name, beer_df in beers.items():
        # Adiciona o DataFrame da cerveja atual à lista auxiliar "dfs"
        dfs.append(beer_df)

# Concatena verticalmente todos os DataFrames presentes na lista "dfs", criando um único DataFrame
# que contém os reviews de algumas cervejas brasileiras dos principais estilos consumidos no Brasil.
brazilian_beer_reviews = pd.concat(dfs, ignore_index=True)

# Exibe o DataFrame resultante da concatenação, que agora contém os dados combinados de todas as cervejas
# listadas no dicionário "beer_dict".
brazilian_beer_reviews

## **Resultado final**

***Caso você não tenha interesse em saber como a filtragem foi realizada, você pode pular direto para essa subseção e ver o resultado final.***

In [ ]:
# Exibe o resultado final da filtragem dos dados.
brazilian_beer_reviews

# **Avaliando as melhores cervejas**

In [ ]:
# Encontrando as cervejas com maior overall
data.sort_values(by='review_overall', ascending=False)

Percebendo que há muitas cervejas com score 5.0 de overall, pensei em criar uma nova coluna com a média geométrica das colunas review_aroma, review_appearance, review_palate e review_taste.

_Por que uma média geométrica?_
- Médias geométricas punem desvios com muito mais força que médias aritméticas. Usando essa estratégia, talvez possamos encontrar a cervejaria com a nota mais sólida. Assim, talvez tenhamos uma avaliação mais rigorosa.

In [ ]:
reviews = ['review_aroma', 'review_appearance', 'review_palate', 'review_taste']
data['geom_mean'] = sp.stats.mstats.gmean(data[reviews], axis=1).round(2)
display(data)

Agora, ordenamos pela média geométrica.

In [ ]:
data = data.sort_values(by='geom_mean', ascending=False)
display(data)

Algo bem confuso nesse resultado é que algumas cervejas com média geométrica 5 nos parâmetros avaliados não têm média 5 de overall...

Usando datetime na coluna de review_time:

In [ ]:
data['review_time'] = pd.to_datetime(data['review_time'], unit='s')
display(data)

Para ver se a média geométrica tem alguma relação positiva com o overall, resolvi fazer um regplot

In [ ]:
#fazendo uma amostragem pra fazer o plot já que a base é bem grande
data_amostra = data.sample(n=10000, random_state=1)
plt.figure(figsize=(7,7))
sns.regplot(data=data_amostra, x='geom_mean', y='review_overall')
plt.show()

In [ ]:
# agora, vendo a correlação em si
data[['review_overall', 'geom_mean']].corr()

Até que a correlação é forte!

Tentando agora encontrar as melhores cervejarias filtrando o df para mostrar somente as cervejas que receberam tanto overall quanto média geométrica 5/5

In [ ]:
melhores_cervejas = data[(data['review_overall'] == 5) & (data['geom_mean'] == 5)]
                                            # dando display por ordem de id
display(melhores_cervejas.sort_values(by='brewery_id'))

Fazendo um df e depois um gráfico com as cervejas que mais foram perfeitamente avaliadas (5/5 em geom mean e overall)

In [ ]:
contagem_cervejas = melhores_cervejas['beer_name'].value_counts().reset_index()
contagem_cervejas.columns = ['beer_name', 'count']
contagem_cervejas = contagem_cervejas.sort_values(by='count', ascending=False)
display(contagem_cervejas)

In [ ]:
top_cervejas = contagem_cervejas.head(15)

# Criar o gráfico de barras
plt.figure(figsize=(12, 6))
sns.barplot(x='count', y='beer_name', data=top_cervejas, palette='YlOrBr')
plt.xlabel('Número de avaliações 5/5')
plt.ylabel('Nome da cerveja')
plt.title('Top 15 Cervejas com avaliações 5/5 em geom_mean e review_overall')
plt.show()